In [3]:
!pip install openinference.instrumentation.langchain
!pip install unstructured

! pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai langchain-text-splitters faiss-cpu 'arize-phoenix[evals]'


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


%md
## Notes
- **Arize Phoenix** is an open-source observability tool for LLM apps.
- It helps log, evaluate, and trace model inputs and outputs.
- Supports metrics like correctness, hallucination, and toxicity.
- Great for debugging agents and tracking multi-step reasoning.
- Works with LangChain, OpenAI, Hugging Face, and more.


In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from phoenix.otel import register
from openinference.instrumentation.langchain import LangChainInstrumentor
import phoenix as px

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()


import os
openai_api_key = os.environ["OPENAI_API_KEY"]


In [6]:
#This starts the Phoenix observability app 
#locally (usually at http://localhost:6006) and opens a session you can use to log and inspect data.

# 🔥 Launch the Phoenix observability dashboard (usually at http://localhost:6006)
px.launch_app(port=6006)

# 🧠 Register a tracer provider to collect trace data for LLM activity
tracer_provider = register()

# 🔗 Connect LangChain to Phoenix using the tracer provider
# This lets Phoenix observe and log each step taken by a LangChain agent
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.


/usr/local/python/3.12.1/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/usr/local/python/3.12.1/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)
E0000 00:00:1747971208.560827   48841 chttp2_server.cc:1682] UNKNOWN:No address added out of total 1 resolved for '[::]:4317' {created_time:"2025-05-23T03:33:28.560821059+00:00", children:[UNKNOWN:Failed to add any wildcard listeners {created_time:"2025-05-23T03:33:28.560799129+00:00", children:[UNKNOWN:Unable to configure socket {created_time:"2025-05-23T03:33:28.557806286+00:00", fd:86, children:[UNKNOWN:bind: Address already in use (98) {created_time:"2025-05-23T03:33:28.550430478+00:00"}]}, UNKNOWN:Unable to configure socket {created_time:"2025-05-23T03:33:28.560794931+00:00", fd:86, children:[UNKNOWN:bind: Address already in use (98) {created_time:"202

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [7]:
# 1. Create a text splitter that breaks documents into chunks of 1024 characters
# with an overlap of 128 characters between each chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)

# 2. Load all documents from the '../city_data' directory
loader = DirectoryLoader("../city_data")

# 3. Load and split the documents using the defined text splitter
docs = loader.load_and_split(text_splitter=text_splitter)

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Step 1: Initialize the OpenAI embedding model
# This model converts each document chunk into a numerical vector
embeddings = OpenAIEmbeddings()

# Step 2: Create a FAISS vector store from the list of documents
# FAISS (Facebook AI Similarity Search) allows fast similarity search over dense vectors
# 'from_documents' will:
# - Embed each chunk in 'docs'
# - Store them in a FAISS index for fast retrieval
db = FAISS.from_documents(docs, embeddings)

# Step 3 (Optional): Perform a similarity search
# Provide a natural language query to retrieve the most relevant document chunks
query = "Tell me about Houston demographics"
results = db.similarity_search(query)
print(results[0].page_content)


AuthenticationError: Error code: 401 - {'error': {'message': "Incorrect API key provided: OPENAI_A*****************************************************************************************************************************************************************************gMA'. You can find your API key at https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# Convert the FAISS vector store into a retriever object
# This allows it to be used in LangChain pipelines like RetrievalQA or ConversationalRetrievalChain
retriever = db.as_retriever()



In [ ]:
from langchain.tools.retriever import create_retriever_tool

# Create a retriever tool from the FAISS retriever
# This tool can be used by an agent to search through vector-retrieved city data
tool = create_retriever_tool(
    retriever,                       # The retriever object (e.g., FAISS as retriever)
    "search_cities",                # Tool name (how the agent will refer to it)
    "Searches and returns excerpts from Wikipedia entries of many cities."  # Tool description
)

# Add this tool to a list of tools the agent can use
tools = [tool]


In [ ]:
from langchain import hub

# Pull a prebuilt prompt from the LangChain Hub.
# This prompt is specifically designed for use with the "openai-tools" agent type.
# The agent needs a structured system prompt to understand:
# - What tools it has access to (like a retriever, calculator, etc.)
# - How to decide when to use a tool vs. just respond
# - How to format tool calls and interpret tool outputs
prompt = hub.pull("hwchase17/openai-tools-agent")

# Display the list of messages that make up the prompt.
# The prompt typically includes a system message with tool instructions and
# may also include example interactions (e.g., few-shot examples).
# These messages will be fed to the Chat model to guide its behavior.
prompt.messages


/home/codespace/.python/current/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize a ChatOpenAI model instance (like GPT-3.5 or GPT-4) for use in a LangChain workflow
llm = ChatOpenAI(
    temperature=0,   # Controls randomness. 0 = most deterministic and factual (great for QA tasks)
    verbose=True     # If True, LangChain will print out detailed logs of what the LLM is doing
)


In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

# Create a tool-using agent using the OpenAI function-calling (tools) capability.
# - 'llm' is your ChatOpenAI instance
# - 'tools' is a list of tools the agent can use (e.g., your retriever tool)
# - 'prompt' is the system message template that tells the agent how to interact with tools
agent = create_openai_tools_agent(
    llm=llm,        # The language model (e.g., GPT-4)
    tools=tools,    # The tools available to the agent (retriever, calculator, etc.)
    prompt=prompt   # Prompt pulled from LangChain Hub to guide tool usage
)

# Wrap the agent in an AgentExecutor
# The executor is what you’ll actually call to run the agent and handle input/output
# It manages calling the agent, interpreting tool outputs, and looping as needed
agent_executor = AgentExecutor(
    agent=agent,    # The agent logic (how to choose and use tools)
    tools=tools     # Tool definitions (must match what the agent expects)
)


In [ ]:
# Invoke the agent_executor with a user input
# The agent will:
# 1. Read the input question
# 2. Decide whether it needs to use any tools (like your retriever)
# 3. Use those tools to fetch relevant data (if needed)
# 4. Generate a final answer using the LLM

result = agent_executor.invoke(
    {
        "input": "Which city has the best food scene out of Lisbon, Houston, Berlin, Moscow, and Paris?"
    }
)


In [ ]:
result["output"]

'Based on the information retrieved, each city has its own unique food scene:\n\n- **Lisbon**: Lisbon is known for its diverse food and restaurant culture. It is recognized for its seafood dishes, pastries like Pastel de Nata, and traditional Portuguese cuisine.\n\n- **Houston**: Houston is highly regarded for its diverse food scene and is often referred to as the "Culinary Capital of the South." The city offers a wide range of international cuisines due to its diverse population.\n\n- **Berlin**: Berlin has a vibrant food scene with a mix of traditional German cuisine and international influences. The city is known for its street food, multicultural eateries, and trendy food markets.\n\n- **Moscow**: Moscow offers a mix of traditional Russian cuisine and international dining options. The city is home to a variety of restaurants, cafes, and food markets serving both local and global dishes.\n\n- **Paris**: Paris is famous for its culinary heritage and is known as a gastronomic capital.